In [1]:
import torch
import numpy as np
import pytorch_lightning as pl
from torch.utils.data import *
import sys
import os
import datetime
import pickle

from new_utils import *
from network import *
from plmodel import *






def str2bool(s):
    if s not in {'false', 'true'}:
        raise ValueError('Not a valid boolean string')
    return s == 'true'
import argparse
import datetime



parser = argparse.ArgumentParser()


parser.add_argument('--batch_size', default=128, type=int) #mat1(a,b)->a
parser.add_argument('--beam_size', type=int, default=10)


parser.add_argument('--cuda', type=int, default=0)
parser.add_argument('--data_name', default=None, type=str)

parser.add_argument('--dataset',  default='None', type=str)
parser.add_argument('--datasets',  default='taobao', type=str)
parser.add_argument('--device', default='cuda', type=str)
parser.add_argument('--dropout_rate', default=0.25, type=float)
parser.add_argument("--embedding_dim", type=int, default=64,
                     help="using embedding")
parser.add_argument('--hidden_units', default=64, type=int)
parser.add_argument('--incatemaxlen', type=int, default=10)


parser.add_argument('--lastk', type=int, default=5)
parser.add_argument('--lr', default=.001, type=float)
parser.add_argument('--l2_emb', default=0.005, type=float)
parser.add_argument('--maxlen', type=int, default=50)
parser.add_argument('--model', type=str, default='casr')


parser.add_argument('--num_layers', default=2, type=int)
parser.add_argument('--num_epochs', default=100, type=int)
parser.add_argument('--num_heads', default=2, type=int)


parser.add_argument('--optimizer_type', default='Adagrad', type=str)
parser.add_argument('--recall', default=[5, 10, 20, 50], type=list)



parser.add_argument("--seed", type=int, default=5,
                     help="Seed for random initialization")


parser.add_argument("-sigma", type=float, default=None,
                     help="init weight -1: range [-sigma, sigma], -2: range [0, sigma]")
parser.add_argument('--state_dict_path', default=None, type=str)

parser.add_argument('--time_bucket', type=int, default=64)
parser.add_argument('--time_span', type=int, default=64)
parser.add_argument('--topkc', default=1, type=int)


args, _ = parser.parse_known_args()

np.random.seed(args.seed)
if args.device == 'cuda':
    torch.cuda.manual_seed(args.seed)
    torch.cuda.manual_seed_all(args.seed)

def init_model(model):
    if args.sigma is not None:
        for p in model.parameters():
            if args.sigma != -1 and args.sigma != -2:
                sigma = args.sigma
                p.data.uniform_(-sigma, sigma)
            elif len(list(p.size())) > 1:
                sigma = np.sqrt(6.0 / (p.size(0) + p.size(1)))
                if args.sigma == -1:
                    p.data.uniform_(-sigma, sigma)
                else:
                    p.data.uniform_(0, sigma)

def count_parameters(model):
    parameter_num = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print("parameter_num", parameter_num) 


batch_size = args.batch_size
lr = args.lr




st = datetime.datetime.now()

if args.datasets =='sub_taobao':
    args.dataset = './data/subtb_10_50.csv'
    User = dataall_partition_last(args.dataset)


if args.datasets =='ijcai':
    args.dataset = '/dev-data/zzz/207/multibehavior/dataset/ijcai/mergedijcai_10_300.csv'
    User = dataall_partition_last(args.dataset)
    args.batch_size = 64

if args.datasets =='retail':
    args.dataset = '/dev-data/zzz/207/multibehavior/dataset/Retail/mergedmultiretail_10_300.csv'
    # args.lr = 0.001
    User = dataall_partition_last(args.dataset)
    args.batch_size = 512


if args.datasets =='smalltaobao':
    args.dataset = '/dev-data/zzz/207/multibehavior/dataset/taobao/mergedmultismalltb_10_40.csv'
    User = dataall_partition_last(args.dataset)

if args.datasets =='taobao':
    args.dataset = '/dev-data/zzz/207/multibehavior/dataset/taobao/mergedmultitaobao.csv'
    User = dataall_partition_last(args.dataset)
    args.batch_size = 128

directory = './dataset/'+args.datasets+'/'

if not os.path.exists(directory):
    os.mkdir(directory)  # 如果目录不存在，则创建它
    print(f"'{directory}' created" )
else:
    print(f" '{directory}' exists")

item_res_train, cate_res_train,  item_res_valid, cate_res_valid, item_res_test, cate_res_test,  usernum,  itemnum, catenum= datatimenormalizelast(User)

num_batch = len(item_res_train) // args.batch_size
cc = 0.0
for u in item_res_train:
    cc += len(item_res_train[u])
print('average item sequence length: %.2f' % (cc / len(item_res_train)))



item_res_all = dict()
cate_res_all = dict()

item_res_eval = dict()
cate_res_eval = dict()
for user in range(1,usernum+1):
    item_res_all[user] = item_res_train[user] + item_res_valid[user] + item_res_test[user]
    cate_res_all[user] = cate_res_train[user] + cate_res_valid[user] + cate_res_test[user]

for user in range(1,usernum+1):
    item_res_eval[user] = item_res_train[user] + item_res_valid[user] 
    cate_res_eval[user] = cate_res_train[user] + cate_res_valid[user] 



# time_point = generate_timepointfromtime(time_res_all, args.time_bucket)



et = datetime.datetime.now()
print("duration ", et-st)
dataloader = lastslow_newDataloader()
try:
    print('try')
    with open('./dataset/%s/dataloader_%d_train%d.pickle'%(args.datasets, args.maxlen,  args.batch_size),"rb") as f:
        wswe= pickle.load(f)
    dataloader.load(wswe, args)

except Exception as e:
    print(e.args)
    print('init1')
    dataloader.init(item_res_train, cate_res_train,  usernum, args.batch_size, 5, args.maxlen, args.incatemaxlen)

    with open('./dataset/%s/dataloader_%d_train%d.pickle'%(args.datasets, args.maxlen, args.batch_size),"wb") as f:
        pickle.dump(dataloader.dataset, f)

[usetensors, itemytensors, cateytensors,  itemseqtensors, cateseqtensors, padincateitemtensors, padcatesettensors, itemyseqtensors, cateyseqtensors] = zip(*dataloader.dataset)

traindataset = mydataset(usetensors, itemytensors, cateytensors, itemseqtensors, cateseqtensors,  padincateitemtensors,  padcatesettensors, itemyseqtensors, cateyseqtensors, args)

evaldataloader = lastslow_newevalDataloader()
try:
    print('try')
    with open('./dataset/%s/dataloader_%d_eval%d.pickle'%(args.datasets, args.maxlen, args.batch_size),"rb") as f:
        wswe= pickle.load(f)
    evaldataloader.load(wswe, args)

except Exception as e:
    print(e.args)
    print('init2')
    evaldataloader.init(item_res_eval, cate_res_eval,  item_res_valid, usernum, args.batch_size, 5, args.maxlen, args.incatemaxlen)
    with open('./dataset/%s/dataloader_%d_eval%d.pickle'%(args.datasets, args.maxlen, args.batch_size),"wb") as f:
            pickle.dump(evaldataloader.dataset, f)

[usetensors, itemytensors, cateytensors, itemseqtensors, cateseqtensors, padincateitemtensors, padcatesettensors,itemyseqtensors, cateyseqtensors] = zip(*evaldataloader.dataset)

evaldataset = mydataset(usetensors, list(itemytensors), list(cateytensors), itemseqtensors, cateseqtensors, padincateitemtensors, padcatesettensors, itemyseqtensors,  cateyseqtensors,  args)

testdataloader = lastslow_newevalDataloader()
try:
    print('try')
    with open('./dataset/%s/dataloader_%d_test%d.pickle'%(args.datasets, args.maxlen, args.batch_size),"rb") as f:
        wswe= pickle.load(f)
    testdataloader.load(wswe, args)

except Exception as e:
    print(e.args)
    print('init2')
    testdataloader.init(item_res_all, cate_res_all, item_res_test, usernum, args.batch_size, 5, args.maxlen, args.incatemaxlen)
    with open('./dataset/%s/dataloader_%d_test%d.pickle'%(args.datasets, args.maxlen, args.batch_size),"wb") as f:
            pickle.dump(testdataloader.dataset, f)







[usetensors, itemytensors, cateytensors, itemseqtensors, cateseqtensors, padincateitemtensors, padcatesettensors,itemyseqtensors, cateyseqtensors] = zip(*testdataloader.dataset)

testdataset = mydataset(usetensors, list(itemytensors), list(cateytensors), itemseqtensors, cateseqtensors, padincateitemtensors, padcatesettensors, itemyseqtensors,  cateyseqtensors,  args)


mydata = DataModule(traindataset, evaldataset, testdataset, args)



# imap, imaps = catemap(args.dataset)
# args.imap = imap



# if args.model =='casr':
    
#     itemmodel = cudaSASRec( itemnum, args)
#     catemodel = cudatqSASRec( catenum, args )
#     recmodel = casrmodel(itemmodel, catemodel, args)


# trainer = pl.Trainer(devices=[args.cuda], accelerator="gpu", accumulate_grad_batches=1, max_epochs=args.num_epochs,  default_root_dir='./script/{}/{}//beamsize{}/lr{}'.format(args.model, args.datasets, args.beam_size,  args.lr))
# trainer.fit(recmodel, mydata)




Preparing data...
 './dataset/taobao/' exists
average item sequence length: 52.85
duration  0:00:12.928361
try
try
try


In [2]:
for user_seq_batch, item_y_batch, cate_y_batch,  item_seq_batch, cate_seq_batch,  pad_incateitem_seq_batch, pad_cateset_seq_batch, itemy_seq_batch, catey_seq_batch in mydata.traindataset:
    break

In [6]:
user_seq = torch.tensor(user_seq_batch)

In [9]:
# 生成二进制掩码
mask = torch.bernoulli(torch.full(user_seq.shape, 0.1))
# 生成随机替换值
replacement = torch.randint(1, usernum+1, user_seq.shape)
mask, replacement

(tensor([0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         1., 1.]),
 tensor([51850, 45514, 48639, 25866, 13331, 30206,  3024, 31317, 18109,  4824,
         27880,  9003, 41395, 18348,  5686, 44151, 44160, 37220, 26206, 14883,
          2847, 31805, 52215, 11171, 28666, 39385,  6638, 25496, 22889, 36314,
         24412, 40670, 28384, 53218, 30279, 41665, 45103,  3445,  3289, 38806,
         13999, 53069, 21912, 45282, 23769, 27095,   609, 32668,  7809, 17531,
         38397,  56

In [ ]:
torch.where()

In [11]:

new_tensor = torch.where(mask.bool(), replacement, user_seq)
new_tensor

tensor([43956, 44623, 48312, 25866, 13331, 47444, 47448, 47883, 47944, 48101,
        48144, 48693, 48844, 49010, 49210, 49282, 46436, 46686, 46704, 46788,
        47141, 47248, 47599, 47838, 28666, 39385, 47935, 47960, 48012, 48055,
        48079, 48083, 48137, 48222, 30279, 48334, 48337, 48432, 48433, 48496,
        48506, 53069, 48530, 45282, 48585, 48691, 48712, 48773, 48835, 48856,
        48932,  5686, 49012, 49023, 49050, 49118, 49132, 49178, 49190, 24670,
        49215, 17003, 49246, 49316, 42247, 43929, 44655, 53701, 44877, 45138,
        45238, 45640, 31879, 45816, 29572, 46234, 46302, 46355, 46376, 31491,
        46426, 46492, 46496, 46510, 46543, 20066, 46629, 46646, 26060, 35134,
        46687, 46693, 46781, 46822, 46835, 46895, 46899, 46913, 46982, 46993,
        47092, 47119, 47135, 47158, 47262, 47330, 47333, 47401, 47438, 47493,
        47511, 47512, 47519, 47578, 47607, 47616, 47622, 47700, 47735, 47761,
        47839, 47840, 47841, 47842, 43856, 47844,  2561, 17416])

In [ ]:
import torch

# 创建一个大小为 [256, 50] 的随机tensor
tensor = torch.randn(256, 50)

# 生成二进制掩码
mask = torch.bernoulli(torch.full(tensor.shape, 0.1))

# 生成随机替换值
replacement = torch.randn(tensor.shape)

# 使用 torch.where 替换元素
new_tensor = torch.where(mask.bool(), replacement, tensor)